In [ ]:
import pickle

In [ ]:
filename = "../CodeSearchNet/big_csn_data/CodeSearchNet_Java_Search_Preliminary.pkl"
infile = open(filename,'rb')
new_dict = pickle.load(infile)
infile.close()

In [ ]:
len(new_dict)

In [ ]:
i = -1 #+(100000)*15
print(new_dict[i]['function'])

In [ ]:
len(new_dict[i]['function_tokens'])

## Analyzing Compression

In [1]:
import json
import copy

In [2]:
def getJsonData(JsonFile):
    with open(JsonFile, encoding="utf8") as f:
        data = json.load(f)
    return data

In [3]:
def getJsonDataLineByLine(JsonFile):
    data = []
    i = 0
    with open(JsonFile) as f:
        for line in f:
            i += 1
            line = line.strip()
            if line[0] == '[':
                line = line[1:]
            elif line[-1] == ']':
                line = line[:-1]
            if line[-1] == ',':
                line = line[:-1]
            try:
                data.append(json.loads(line))
                if i%10000 == 0: print(i, "success")
            except:
                data.append('''{"classBodyDeclaration": ""}''')
                print(i, "failure")
                print(line)
    return data

In [4]:
def getJsonDataSafe(JsonFile):
    try:
        return getJsonData(JsonFile)
    except Exception as e:
        print(e)
        print("Loading JSON failed. Attempting safe method.")
        return getJsonDataLineByLine(JsonFile)

In [5]:
def dump_dict(data, name):
    with open(name, 'w', encoding="utf8") as f:
        json.dump(data, f)

# Visualize Tree

In [6]:
def travarse(tree):
    if 'type' in tree:
        print(tree['text'], end=" ")
        if tree['text'] in [';', '{', '}']:
            print()
        return
    else:
        root = list(tree.keys())[0]
        children = tree[root]
        for child in children:
            travarse(child)

In [85]:
import ast
def tree2code2(tree, indentation):
    parent = list(tree.keys())[0]
    children = tree[parent]
    if 'type' in tree:
        code = tree['text'] + ' '
        if tree['text'] in [';', '{', '}']:
            code += '\n'
            if tree['text'] == '{':
                indentation += 1
            elif tree['text'] == '}':
                indentation -= 1
            code += '\t'*indentation
        #return code, indentation
    elif len(children)==1 and parent == "literal":
        child = children[0]
        child_text = child["text"]
        type(ast.literal_eval(child_text))
        try:
            if child_text in ['null', 'true', 'false']:
                pass
            elif type(ast.literal_eval(child_text)) == type("STRING"):
                code += "STRING00"
            elif type(ast.literal_eval(child_text)) == type(1):
                code += "INT00"
            elif type(ast.literal_eval(child_text)) == type(1.1):
                code += "FLOAT00"
        except:
            #code += "NUM00"
            pass
    else:
        root = list(tree.keys())[0]
        children = tree[root]
        code = ""
        for child in children:
            t, indentation = tree2code2(child, indentation)
            code += t
    return code, indentation

In [86]:
for i in range(100):
    print(tree2code(tree_data[i],0)[0])

public OmemoDevice getOwnDevice ( ) { 
	synchronized ( LOCK ) { 
		BareJid jid = getOwnJid ( ) ; 
		if ( jid == null ) { 
			return null ; 
			} 
		return new OmemoDevice ( jid , getDeviceId ( ) ) ; 
		} 
	} 

void setDeviceId ( int nDeviceId ) { 
	synchronized ( LOCK ) { 
		INSTANCES . get ( connection ( ) ) . remove ( getDeviceId ( ) ) ; 
		INSTANCES . get ( connection ( ) ) . put ( nDeviceId , this ) ; 
		this . deviceId = nDeviceId ; 
		} 
	} 

void notifyOmemoMessageReceived ( Stanza stanza , OmemoMessage . Received decryptedMessage ) { 
	for ( OmemoMessageListener l : omemoMessageListeners ) { 
		l . onOmemoMessageReceived ( stanza , decryptedMessage ) ; 
		} 
	} 

void notifyOmemoMucMessageReceived ( MultiUserChat muc , Stanza stanza , OmemoMessage . Received decryptedMessage ) { 
	for ( OmemoMucMessageListener l : omemoMucMessageListeners ) { 
		l . onOmemoMucMessageReceived ( muc , stanza , decryptedMessage ) ; 
		} 
	} 

void notifyOmemoCarbonCopyReceived ( CarbonExtension . 

In [8]:
def printCode(tree):
    code, _ = tree2code(tree, 0)
    print(code)
    print("Count:", len(code.split()))

In [9]:
ls ../csn_big_only_trees/

Parameter format not correct - "csn_big_only_trees".


In [ ]:
uncompressed_data = getJsonData('../csn_big_only_trees/csn_big_1.json')

In [ ]:
compressed_data = getJsonData('../CodeSearchNet/train_test_valid_data_10k/java_train_1u_10k_compressed.json')

In [ ]:
bef_comp = 0
aft_comp = 0
for i in range(len(compressed_data)):
    code, _ = tree2code(uncompressed_data[i], 0)
    num_tokens = len(code.split())
    compressed_tokens = len(compressed_data[i]['code'].split())
    bef_comp += num_tokens
    aft_comp += compressed_tokens
    if not compressed_tokens:
        aft_comp += num_tokens

print(1-aft_comp/bef_comp)

In [ ]:
i = 205

In [ ]:
code, _ = tree2code(uncompressed_data[i], 0)
num_tokens = len(code.split())
print(code)
print(num_tokens)
print("================================")
print(compressed_data[i]['code'])
compressed_tokens = len(compressed_data[i]['code'].split())
print(compressed_tokens)
print("Compression Ratio:", compressed_tokens/num_tokens)
i+=1

In [ ]:
0.2104572106016639

## How many of the compressed tokens were present in the top 10k train data?

In [ ]:
abstract_code_dict = getJsonData('../CodeSearchNet/abstract_code_dict_500k_w_child.json')

In [ ]:
abstract_code_keys = sorted(abstract_code_dict, key = lambda x:abstract_code_dict[x]['count'], reverse=True)

In [ ]:
abstract_code_dict[abstract_code_keys[110]]

In [ ]:
most_freq_25k = [abstract_code_dict[abstract_code_keys[i]]['compressed_name'] for i in range(25000)]

In [ ]:
most_freq_10k = [abstract_code_dict[abstract_code_keys[i]]['compressed_name'] for i in range(10000)]

In [ ]:
most_freq_25k[:1000]

In [ ]:
#!mkdir ../CodeSearchNet/25k
#!mv ../CodeSearchNet/train-test-valid_25kf.zip ../CodeSearchNet/25k/
#!unzip ../CodeSearchNet/25k/train-test-valid_25kf.zip
!ls ../CodeSearchNet/train_test_valid_data_10k/

In [ ]:
import glob
dirs = glob.glob('../CodeSearchNet/train_test_valid_data_10k_500k/*_train_*')
dirs

In [ ]:
freq_dict = {}
for path in dirs:
    dataset = getJsonData(path)
    for data in dataset:
        code = data['code']
        code_tokens = code.split()
        for token in code_tokens:
            if token not in freq_dict:
                freq_dict[token] = 0
            freq_dict[token] += 1

In [ ]:
len(freq_dict)

In [ ]:
# 25k compressed total tokens: 1024352
# 10k compressed total tokens: 1160247

In [ ]:
data_freq_list = sorted(freq_dict, key = lambda x:freq_dict[x], reverse=True)

In [ ]:
top_data_freq_list = data_freq_list[:30000]

In [ ]:
top_data_freq_list[:1000]

In [ ]:
common_tokens = list(set(top_data_freq_list) & set(most_freq_10k))

In [ ]:
len(common_tokens)

In [ ]:
len(top_data_freq_list)

Dumping common union vocabs

In [ ]:
list_10k_10k = []

In [ ]:
for token in top_data_freq_list:
    list_10k_10k.append({'token':token, 'count':freq_dict[token]})

In [ ]:
additional_compressed_tokens = list(set(most_freq_10k) - set(top_data_freq_list))
len(additional_compressed_tokens)

In [ ]:
for token in additional_compressed_tokens:
    if token in freq_dict:
        list_10k_10k.append({'token':token, 'count':freq_dict[token]})

In [ ]:
len(list_10k_10k)

In [ ]:
dump_dict(list_10k_10k, "list_10k_10k.json")

Testing out data outside vocab

In [ ]:
not_in_vocab = list(set(most_freq_10k) - set(top_data_freq_list))
len(not_in_vocab)

In [ ]:
not_in_vocab[:100]

In [ ]:
for f in not_in_vocab:
    print(f,"||", func2abs[f],"||", abstract_code_dict[func2abs[f]]['count'])

In [ ]:
for f in common_tokens[:1000]:
    print(f,"||", func2abs[f],"||", abstract_code_dict[func2abs[f]]['count'])

In [ ]:
abstract_code_dict[abstract_code_keys[10000]]

In [ ]:
abstract_code_dict['( ( AbstractISUPParameter ) $id0 ) . decode ( $id1 ) ;']

In [ ]:
'$F2570015' in common_tokens

In [ ]:
len(abstract_code_keys)
#1144264

In [ ]:
i = -1
abstract_code_keys2 = []
for key in abstract_code_keys:
    i+=1
    parent = key + " ;" 
    if parent in abstract_code_keys:
        if not i%1000: print(i)
        if abstract_code_dict[key]['count'] != abstract_code_dict[parent]['count']:
            abstract_code_keys2.append(key)

abstract_code_keys = abstract_code_keys2
len(abstract_code_keys)

In [ ]:
i = -1
abstract_code_keys2 = []
for key in abstract_code_keys:
    i+=1
    if not i%100: print(i)
    #abstract_code_keys.remove(key)
    abstract_code_keys2.append(key)

## Removing child counts

In [ ]:
min_freq = freq_dict[data_freq_list[30000]]
min_freq

In [ ]:
for discard_after in range(len(abstract_code_keys)):
    key = abstract_code_keys[discard_after]
    if abstract_code_dict[key]['count'] <= min_freq:
        break
discard_after

In [ ]:
func2abs = {}

for key in abstract_code_dict:
    func2abs[abstract_code_dict[key]['compressed_name']] = key

In [ ]:
accepted_abstract_keys = abstract_code_keys[:discard_after]
abstract_code_dict_modified = copy.deepcopy(abstract_code_dict)

In [ ]:
for key in accepted_abstract_keys:
    children = abstract_code_dict[key]['children']
    if children:
        count = abstract_code_dict[key]['count']
        for child in children:
            try:
                child_key = func2abs[child]
            except:
                print("parent:", abstract_code_dict_modified[key], "child:", child)
            abstract_code_dict_modified[child_key]['count'] -= count

In [ ]:
abstract_code_keys = sorted(abstract_code_dict_modified, key = lambda x:abstract_code_dict_modified[x]['count'], reverse=True)

In [ ]:
for discard_after in range(len(abstract_code_keys)):
    key = abstract_code_keys[discard_after]
    if abstract_code_dict_modified[key]['count'] <= min_freq:
        break
discard_after

In [ ]:
accepted_abstract_keys = abstract_code_keys[:discard_after]

In [ ]:
len(top_data_freq_list)

In [ ]:
abstract_code_dict2 = getJsonData('../CodeSearchNet/abstract_code_dict_500k.json')

In [ ]:
accepted_abstract_funcs = [ abstract_code_dict2[x]['compressed_name'] for x in accepted_abstract_keys ]

In [ ]:
not_in_vocab = list(set(accepted_abstract_funcs) & set(top_data_freq_list))
len(not_in_vocab)

## Testing new model

In [ ]:
## Loading one training data
tree_data = getJsonData("../test_data/java_train_3.json")

In [11]:
## build freq_dict
freq_dict = {}
uncompressed_codes = []

for i in range(len(tree_data)):
    if i%5000 == 0: print("[", i, "]")
    code, _ = tree2code(tree_data[i], 0)
    uncompressed_codes.append({"code": code})
    tokens = code.split()

    for token in tokens:
        if token not in freq_dict: freq_dict[token] = 0
        freq_dict[token] += 1

[ 0 ]
[ 5000 ]
[ 10000 ]
[ 15000 ]
[ 20000 ]
[ 25000 ]


In [13]:
dump_dict(uncompressed_codes, "../test_data/uncompressed_codes_train_3.json")

In [14]:
import ast
def get_compressed_func():
    global compression_number
    compression_number += 1
    return "$F"+str(compression_number)

In [27]:
def build_dict(tree):
    var_list = []
    global abstract_code_dict
    global num_to_compress
    global function2abstract
    
    if 'text' in tree:
        code = tree["text"] + " "
        #print("**:\t",code, 1)
        return (code, 1, 0, [], None)
    else:
        parent = list(tree.keys())[0]
        children = tree[parent]
        if len(children) == 0:
            return ("", 0, 0, [], None)
        
        code = ""
        compressed_func = None
        leaf_count = 0
        id_count = 0
        children_list = []
        #leaf_list = []
        
        '''
        and not (len(children) == 2 and 'text' in children[1] and children[1]['text'] == ';')\
        and not (len(children) == 3 and 'text' in children[0] and 'text' in children[2] and \
                 ((children[0]['text'] == '(' and children[2]['text'] == ')') or 
                  (children[0]['text'] == '{' and children[2]['text'] == '}') or
                  (children[0]['text'] == '[' and children[2]['text'] == ']')))
        '''
        
        if len(children) > 1:
            for child in children:
                (c, n, i, vl, f) = build_dict(child)
                code += c
                leaf_count += n
                id_count += i
                var_list += [v for v in vl if v not in var_list]
                if f: children_list.append(f)
                #else: leaf_list += code.split()
                
            if (var_list and leaf_count - len(var_list) >= 4) or (not var_list and leaf_count > 1) :
                abstract_code = code.split()
                for i in range(len(abstract_code)):
                    if abstract_code[i] in var_list:
                        abstract_code[i] = "$id{}".format(var_list.index(abstract_code[i]))
                abstract_code = ' '.join(abstract_code)
                
                if abstract_code not in abstract_code_dict:
                    compressed_func = get_compressed_func()
                    abstract_code_dict[abstract_code] = {
                        'count': 0, 
                        'compressed_name': compressed_func,
                        'num_params': len(var_list),
                        'children': children_list,
                        #'leaves': leaf_list
                    }
                    function2abstract[compressed_func] = abstract_code
                else:
                    compressed_func = abstract_code_dict[abstract_code]['compressed_name']
                
                abstract_code_dict[abstract_code]['count'] += 1
#                 children_list.insert(0, compressed_func)
                
            return (code, leaf_count, id_count, var_list, compressed_func)
        
        if parent == "literal":
            child = children[0]
            child_text = child["text"]
            try:
                if child_text in ['null', 'true', 'false']:
                    pass
                elif type(ast.literal_eval(child_text)) == type("STRING"):
                    children[0]["text"] = "STRING00"
                elif type(ast.literal_eval(child_text)) == type(1):
                    children[0]["text"] = "INT00"
                elif type(ast.literal_eval(child_text)) == type(1.1):
                    children[0]["text"] = "FLOAT00"
            except:
                children[0]["text"] = "NUM00"
            code = children[0]["text"] + " "
#==================================================================
        elif parent == "primary" and 'type' in children[0]:
            id_count += 1
            text = children[0]["text"]
            if text not in var_list:
                var_list.append(text)
            code = text + " "
        elif parent == "variableDeclaratorId" and 'type' in children[0]:
            id_count += 1
            text = children[0]["text"]
            if text not in var_list:
                var_list.append(text)
            code = text + " "
#=================================================================
        else:
            (code, leaf_count, i, vl, compressed_func) = build_dict(children[0])
            id_count += i
            var_list += [v for v in vl if v not in var_list]
        
        return (code, leaf_count, id_count, var_list, compressed_func)

### Building Dictionary
[Parent Child](#Parent-child)

In [28]:
abstract_code_dict = {}
compression_number = 0
function2abstract = {}

In [29]:
for data in tree_data:
    build_dict(data)

In [31]:
abstract_code_keys = sorted(abstract_code_dict, key = lambda x:abstract_code_dict[x]['count'], reverse=True)
len(abstract_code_keys)

411217

In [32]:
for key in abstract_code_keys[:40]:
    print(key, "-->", abstract_code_dict[key])

( ) --> {'count': 13831, 'compressed_name': '$F1', 'num_params': 0, 'children': []}
@ Override --> {'count': 5896, 'compressed_name': '$F178', 'num_params': 0, 'children': []}
( NUM00 ) --> {'count': 5555, 'compressed_name': '$F84', 'num_params': 0, 'children': []}
IllegalArgumentException ( NUM00 ) --> {'count': 3727, 'compressed_name': '$F2241', 'num_params': 0, 'children': ['$F84']}
new IllegalArgumentException ( NUM00 ) --> {'count': 3727, 'compressed_name': '$F2242', 'num_params': 0, 'children': ['$F2241']}
throw new IllegalArgumentException ( NUM00 ) ; --> {'count': 3726, 'compressed_name': '$F2243', 'num_params': 0, 'children': ['$F2242']}
break ; --> {'count': 3654, 'compressed_name': '$F1188', 'num_params': 0, 'children': []}
{ throw new IllegalArgumentException ( NUM00 ) ; } --> {'count': 3567, 'compressed_name': '$F2244', 'num_params': 0, 'children': ['$F2243']}
return $id0 . body ( ) ; --> {'count': 2244, 'compressed_name': '$F354778', 'num_params': 1, 'children': []}
{ ret

In [19]:
most_freq = sorted(freq_dict, key = lambda x:freq_dict[x], reverse=True)

In [34]:
func2abs = {}

for key in abstract_code_dict:
    func2abs[abstract_code_dict[key]['compressed_name']] = key

In [ ]:
vocab_size = 10000

In [ ]:
min_freq = freq_dict[most_freq[vocab_size]]
most_freq[vocab_size], min_freq

In [ ]:
for key in list(abstract_code_dict):
    if abstract_code_dict[key]['count'] < min_freq:
        del abstract_code_dict[key]

In [ ]:
for key in abstract_code_dict:
    children = abstract_code_dict[key]['children']
    count = abstract_code_dict[key]['count']
    for child in children:
        child_key = func2abs[child]
        if child_key in abstract_code_dict:
            abstract_code_dict[child_key]['count'] -= count

In [21]:
abstract_code_keys = sorted(abstract_code_dict, key = lambda x:abstract_code_dict[x]['count'], reverse=True)
len(abstract_code_keys)

399882

In [ ]:
for x in abstract_code_keys:
    

In [ ]:
for key in abstract_code_dict:
    func = abstract_code_dict[key]['compressed_name']
    freq_dict[func] = abstract_code_dict[key]['count']

In [ ]:
i = 0
for key in abstract_code_keys:
    if not i%100: print(i)
    i+=1
    most_freq = sorted(freq_dict, key = lambda x:freq_dict[x], reverse=True)
    count = abstract_code_dict[key]['count']
    func = abstract_code_dict[key]['compressed_name']
    if func in most_freq[:vocab_size]:
        tokens = key.split()
        for token in tokens:
            if not token.startswith("$id") if token in freq_dict:
                freq_dict[token] -= count

In [ ]:
freq_dict[most_freq[vocab_size]]

In [ ]:
to_be_compressed = []
for token in most_freq[:vocab_size]:
    if token.startswith("$F"):
        to_be_compressed.append(func2abs[token])

In [ ]:
len(to_be_compressed), to_be_compressed[0]

In [ ]:
abstract_code_keys = to_be_compressed

In [ ]:
def compress(tree):
    var_list = []
    global abstract_code_dict
    global num_to_compress
    
    if 'text' in tree:
        code = tree["text"] + " "
        #print("**:\t",code, 1)
        return (code, 1, 0, var_list)
    else:
        parent = list(tree.keys())[0]
        children = tree[parent]
        if len(children) == 0:
            return ("", 0, 0, [])
        
        code = ""
        leaf_count = 0
        id_count = 0
        
        if len(children) > 1:
            for child in children:
                (c, n, i, vl) = compress(child)
                code += c
                leaf_count += n
                id_count += i
                var_list += [v for v in vl if v not in var_list]

            if (var_list and leaf_count - len(var_list) >= 4) or (not var_list and leaf_count > 1) :
                abstract_code = code.split()
                for i in range(len(abstract_code)):
                    if abstract_code[i] in var_list:
                        abstract_code[i] = "$id{}".format(var_list.index(abstract_code[i]))
                abstract_code = ' '.join(abstract_code)
                
                # If the abstracted code is within the range, replace
                # the children with a single leaf node containing whole code
                # If the list is empty, only one token.
                if abstract_code in abstract_code_keys:
                    compressed_func = abstract_code_dict[abstract_code]['compressed_name']
                    compressed_func += ' ( ' +' , '.join(var_list)+' )' if var_list else ''
                    tree[parent] = [{"type": "compressed", "text": compressed_func}]

            #print(parent, ":\t",code, leaf_count, id_count, var_list)
            return (code, leaf_count, id_count, var_list)

        if parent == "literal":
            child = children[0]
            child_text = child["text"]
            try:
                if child_text in ['null', 'true', 'false']:
                    pass
                elif type(ast.literal_eval(child_text)) == type("STRING"):
                    children[0]["text"] = "STRING00"
                elif type(ast.literal_eval(child_text)) == type(1):
                    children[0]["text"] = "INT00"
                elif type(ast.literal_eval(child_text)) == type(1.1):
                    children[0]["text"] = "FLOAT00"
            except:
                children[0]["text"] = "NUM00"
        elif parent == "primary" and 'type' in children[0]:
            id_count += 1
            text = children[0]["text"]
            if text not in var_list:
                var_list.append(text)
        elif parent == "variableDeclaratorId" and 'type' in children[0]:
            id_count += 1
            text = children[0]["text"]
            if text not in var_list:
                var_list.append(text)

        (code, leaf_count, i, vl) = compress(children[0])
        id_count += i
        var_list += [v for v in vl if v not in var_list]
        
        #print(parent, ":\t",code, leaf_count, id_count, var_list)
        return (code, leaf_count, id_count, var_list)
    

In [ ]:
compressed_freq_dict = {}
compressed_codes = []
for i in range(len(tree_data)):
    if i%1000 == 0: print("[", i, "]") 
    try:
        tree_copy = copy.deepcopy(tree_data[i])
        (code, leaf_count, id_count, var_list) = compress(tree_copy)
        code, _ = tree2code(tree_copy, 0)
    except Exception as e:
        # Write the data number and file name to another file for later investigation
        print(e)
        code, _ = tree2code(tree_data[i], 0)
    compressed_code = {"code": code}
    tokens = code.split()
    for token in tokens:
        if token not in compressed_freq_dict:
            compressed_freq_dict[token] = 0
        compressed_freq_dict[token] += 1
    compressed_codes.append(compressed_code)

In [ ]:
compressed_most_freq = sorted(compressed_freq_dict, key = lambda x:compressed_freq_dict[x], reverse=True)

In [ ]:
compressed_funcs = [abstract_code_dict[x]['compressed_name'] for x in abstract_code_keys]

In [ ]:
common_tokens_after_compression = list(set(compressed_most_freq[:vocab_size]) & set(compressed_funcs))
len(common_tokens_after_compression)

In [ ]:
'$F280' in compressed_funcs

In [ ]:
compressed_most_freq[:100]

In [ ]:
oov = list(set(compressed_funcs) - set(compressed_most_freq[:vocab_size]))
len(oov)

In [ ]:
for o in oov:
    a = abstract_code_dict[func2abs[o]]
    print(o, a[])

In [ ]:
len(abstract_code_keys)

### Parent child
[Dictionary Building](#Building-Dictionary)

In [52]:
count = 0
# IF A CHILD IS BEFORE A PARENT
for key in abstract_code_keys[:100000]:
    children = abstract_code_dict[key]['children']
    for child in children:
        if child in func2abs:
            child_key = func2abs[child]
            
            if set(child_key.split()) - set(key.split()):
                print(count, "::", key, "||", child_key)
                print(abstract_code_dict[key]['count'],"--", abstract_code_dict[child_key]['count'])
                count+=1

0 :: $id0 = $id1 != null ? $id1 . $id0 ( ) : null || $id0 != null ? $id0 . acceptLanguage ( ) : null
339 -- 11
1 :: $id0 = $id1 . body ( ) . $id0 ( ) || $id0 . body ( ) . nextPageLink ( )
181 -- 166
2 :: if ( $id0 == null ) { $id1 . $id0 = null ; } else { $id1 . $id0 = new DateTimeRfc1123 ( $id0 ) ; } || { $id0 . ocpDate = null ; }
152 -- 59
3 :: $id3 < ServiceResponse < Void > > call ( Response < ResponseBody > $id0 ) { try { ServiceResponse < Void > $id1 = $id2 ( $id0 ) ; return $id3 . just ( $id1 ) ; } catch ( Throwable $id4 ) { return $id3 . error ( $id4 ) ; } } || Observable < ServiceResponse < Void > >
59 -- 205
4 :: Func1 < Response < ResponseBody > , $id3 < ServiceResponse < Void > > > ( ) { @ Override public $id3 < ServiceResponse < Void > > call ( Response < ResponseBody > $id0 ) { try { ServiceResponse < Void > $id1 = $id2 ( $id0 ) ; return $id3 . just ( $id1 ) ; } catch ( Throwable $id4 ) { return $id3 . error ( $id4 ) ; } } } || Func1 < Response < ResponseBody > , Observab

33 :: if ( $id0 ( ) == null ) { if ( $id1 . $id0 ( ) != null ) { return false ; } } else if ( ! $id0 ( ) . equals ( $id1 . $id0 ( ) ) ) { return false ; } || { if ( $id0 . getNodeName ( ) != null ) { return false ; } }
5 -- 1
34 :: $id1 . xml . parsers . SAXParserFactory $id0 = $id1 . xml . parsers . SAXParserFactory . newInstance ( ) || javax . xml . parsers . SAXParserFactory
5 -- 5
35 :: $id0 . registerFactory ( NUM00 , new $id0 . Factory ( ) { @ Override public $id0 getInstance ( String $id1 ) { return new EscapeTransliterator ( NUM00 , NUM00 , NUM00 , NUM00 , NUM00 , true , null ) ; } } ) || NUM00 , new Transliterator . Factory ( ) { @ Override public Transliterator getInstance ( String $id0 ) { return new EscapeTransliterator ( NUM00 , NUM00 , NUM00 , NUM00 , NUM00 , true , null ) ; } }
5 -- 5
36 :: $id3 < ServiceResponse < String > > call ( Response < ResponseBody > $id0 ) { try { ServiceResponse < String > $id1 = $id2 ( $id0 ) ; return $id3 . just ( $id1 ) ; } catch ( Throwable

108 :: Func1 < Response < ResponseBody > , $id3 < ServiceResponse < Project > > > ( ) { @ Override public $id3 < ServiceResponse < Project > > call ( Response < ResponseBody > $id0 ) { try { ServiceResponse < Project > $id1 = $id2 ( $id0 ) ; return $id3 . just ( $id1 ) ; } catch ( Throwable $id4 ) { return $id3 . error ( $id4 ) ; } } } || Func1 < Response < ResponseBody > , Observable < ServiceResponse < Project > > >
3 -- 3
109 :: $id3 < ServiceResponse < List < Image > > > call ( Response < ResponseBody > $id0 ) { try { ServiceResponse < List < Image > > $id1 = $id2 ( $id0 ) ; return $id3 . just ( $id1 ) ; } catch ( Throwable $id4 ) { return $id3 . error ( $id4 ) ; } } || Observable < ServiceResponse < List < Image > > >
3 -- 12
110 :: Func1 < Response < ResponseBody > , $id3 < ServiceResponse < List < Image > > > > ( ) { @ Override public $id3 < ServiceResponse < List < Image > > > call ( Response < ResponseBody > $id0 ) { try { ServiceResponse < List < Image > > $id1 = $id2 ( $id0 

191 :: $id1 . Builder < Integer > $id0 = $id1 . $id0 ( ) || ImmutableSet . Builder < Integer >
2 -- 2
192 :: case $id0 : $id1 . scrollToSide ( $id2 . Side . $id0 , $id3 , $id4 ) ; break ; || $id0 . scrollToSide ( $id1 . Side . RIGHT , $id2 , $id3 ) ;
2 -- 1
193 :: case $id0 : $id1 . scrollViewToSide ( $id2 , $id3 . Side . $id0 , $id4 , $id5 ) ; break ; || $id0 . scrollViewToSide ( $id1 , $id2 . Side . RIGHT , $id3 , $id4 ) ;
2 -- 1
194 :: { if ( $id0 instanceof $id1 ) return ( $id1 ) $id0 ; return $id1 . valueOf ( $id2 ( $id0 ) ) ; } || if ( $id0 instanceof Long ) return ( Long ) $id0 ;
2 -- 2
195 :: lucee . runtime . interpreter . ref . var . $id0 ( $id0 . SCOPE_UNDEFINED ) || lucee . runtime . interpreter . ref . var . Scope
2 -- 4
196 :: if ( $id0 instanceof InvocationTargetException && ( ( InvocationTargetException ) $id0 ) . getTargetException ( ) instanceof $id1 ) throw ( $id1 ) ( ( InvocationTargetException ) $id0 ) . getTargetException ( ) ; || ( $id0 instanceof InvocationTarge

2 -- 2
304 :: $id3 < ServiceResponse < DiagnosticAnalysisInner > > call ( Response < ResponseBody > $id0 ) { try { ServiceResponse < DiagnosticAnalysisInner > $id1 = $id2 ( $id0 ) ; return $id3 . just ( $id1 ) ; } catch ( Throwable $id4 ) { return $id3 . error ( $id4 ) ; } } || Observable < ServiceResponse < DiagnosticAnalysisInner > >
2 -- 6
305 :: Func1 < Response < ResponseBody > , $id3 < ServiceResponse < DiagnosticAnalysisInner > > > ( ) { @ Override public $id3 < ServiceResponse < DiagnosticAnalysisInner > > call ( Response < ResponseBody > $id0 ) { try { ServiceResponse < DiagnosticAnalysisInner > $id1 = $id2 ( $id0 ) ; return $id3 . just ( $id1 ) ; } catch ( Throwable $id4 ) { return $id3 . error ( $id4 ) ; } } } || Func1 < Response < ResponseBody > , Observable < ServiceResponse < DiagnosticAnalysisInner > > >
2 -- 2
306 :: $id3 < ServiceResponse < Page < DiagnosticCategoryInner > > > $id4 ( final String $id0 ) { return $id1 ( $id0 ) . concatMap ( new Func1 < ServiceResponse <

2 -- 11
496 :: Func1 < ServiceResponse < Page < VirtualMachineScaleSetSkuInner > > , $id2 < ServiceResponse < Page < VirtualMachineScaleSetSkuInner > > > > ( ) { @ Override public $id2 < ServiceResponse < Page < VirtualMachineScaleSetSkuInner > > > call ( ServiceResponse < Page < VirtualMachineScaleSetSkuInner > > $id0 ) { String $id1 = $id0 . body ( ) . $id1 ( ) ; if ( $id1 == null ) { return $id2 . just ( $id0 ) ; } return $id2 . just ( $id0 ) . concatWith ( $id3 ( $id1 ) ) ; } } || Func1 < ServiceResponse < Page < VirtualMachineScaleSetSkuInner > > , Observable < ServiceResponse < Page < VirtualMachineScaleSetSkuInner > > > >
2 -- 2
497 :: $id3 < ServiceResponse < SummarizeResultsInner > > call ( Response < ResponseBody > $id0 ) { try { ServiceResponse < SummarizeResultsInner > $id1 = $id2 ( $id0 ) ; return $id3 . just ( $id1 ) ; } catch ( Throwable $id4 ) { return $id3 . error ( $id4 ) ; } } || Observable < ServiceResponse < SummarizeResultsInner > >
2 -- 6
498 :: Func1 < Response 

1 -- 1
619 :: $id2 . jivesoftware . smack . chat . Chat createPrivateChat ( EntityJid $id0 , ChatMessageListener $id1 ) { return $id2 . jivesoftware . smack . chat . ChatManager . getInstanceFor ( $id3 ) . createChat ( $id0 , $id1 ) ; } || org . jivesoftware . smack . chat . Chat
1 -- 1
620 :: { HashMap < Jid , $id2 > $id0 = new HashMap < > ( ) ; $id0 . put ( $id1 . getUser ( ) , $id2 . none ) ; MUCLightChangeAffiliationsIQ $id3 = new MUCLightChangeAffiliationsIQ ( $id4 , $id0 ) ; $id7 $id5 = $id1 . createStanzaCollectorAndSend ( $id3 ) . nextResultOrThrow ( ) ; boolean $id6 = $id5 . getType ( ) . equals ( $id7 . Type . result ) ; if ( $id6 ) { $id8 ( ) ; } } || HashMap < Jid , MUCLightAffiliation > $id0 = new HashMap < > ( ) ;
1 -- 1
621 :: { HashMap < Jid , $id2 > $id0 = new HashMap < > ( ) ; $id0 . put ( $id1 . getUser ( ) , $id2 . none ) ; MUCLightChangeAffiliationsIQ $id3 = new MUCLightChangeAffiliationsIQ ( $id4 , $id0 ) ; $id7 $id5 = $id1 . createStanzaCollectorAndSend ( $id3 ) 

784 :: { if ( $id0 . size ( ) <= NUM00 ) { return false ; } final SortedSet < BillingEvent > $id1 = new TreeSet < BillingEvent > ( ) ; final SortedSet < BillingEvent > $id2 = new TreeSet < BillingEvent > ( ) ; final List < BlockingState > $id3 = $id4 . getBlockingAllForAccount ( $id5 , $id6 ) ; final Iterable < BlockingState > $id7 = $id8 . $id26 ( $id3 , new Predicate < BlockingState > ( ) { @ Override public boolean apply ( final BlockingState $id9 ) { return $id10 . ACCOUNT == $id9 . getType ( ) ; } } ) ; final Map < UUID , List < BlockingState > > $id11 = $id12 ( $id10 . SUBSCRIPTION_BUNDLE , $id3 ) ; final Map < UUID , List < BlockingState > > $id13 = $id12 ( $id10 . SUBSCRIPTION , $id3 ) ; for ( final UUID $id14 : $id15 . keySet ( ) ) { final List < BlockingState > $id16 = $id11 . get ( $id14 ) != null ? $id11 . get ( $id14 ) : $id17 . < BlockingState > of ( ) ; for ( final SubscriptionBase $id18 : $id15 . get ( $id14 ) ) { if ( $id19 . contains ( $id18 . getId ( ) ) ) { continue

1 -- 1
959 :: CompletableFuture < $id6 > queryLinearizable ( Indexed < QueryEntry > $id0 ) { return $id1 ( $id0 ) . thenComposeAsync ( $id2 - > $id3 . appendEntries ( ) . thenApply ( $id4 - > $id2 ) . exceptionally ( $id5 - > $id6 . builder ( ) . withStatus ( $id7 . Status . ERROR ) . withError ( $id8 . Type . QUERY_FAILURE , $id5 . getMessage ( ) ) . build ( ) ) , $id9 . getThreadContext ( ) ) ; } || CompletableFuture < QueryResponse >
1 -- 6
960 :: try { return $id0 . completedFuture ( $id1 . getLogWriter ( ) . append ( $id2 ) ) . thenApply ( $id3 - > <missing ';'> { $id4 . trace ( NUM00 , $id3 ) ; return $id3 ; } ) ; } catch ( StorageException . TooLarge $id6 ) { return $id5 . exceptionalFuture ( $id6 ) ; } catch ( StorageException . OutOfDiskSpace $id6 ) { $id4 . warn ( NUM00 ) ; return $id1 . getServiceManager ( ) . compact ( ) . thenCompose ( $id7 - > $id8 ( $id2 , $id9 + NUM00 ) ) ; } || catch ( StorageException . OutOfDiskSpace e ) { $id0 . warn ( NUM00 ) ; return $id1 . getSer

In [53]:
count

1109

In [42]:
61718/3

20572.666666666668

In [43]:
print(uncompressed_codes[20572]['code'])

static Person PromptForAddress ( BufferedReader stdin , PrintStream stdout ) throws IOException { 
	Person . Builder person = Person . newBuilder ( ) ; 
	stdout . print ( "Enter person ID: " ) ; 
	person . setId ( Integer . valueOf ( stdin . readLine ( ) ) ) ; 
	stdout . print ( "Enter name: " ) ; 
	person . setName ( stdin . readLine ( ) ) ; 
	stdout . print ( "Enter email address (blank for none): " ) ; 
	String email = stdin . readLine ( ) ; 
	if ( email . length ( ) > 0 ) { 
		person . setEmail ( email ) ; 
		} 
	while ( true ) { 
		stdout . print ( "Enter a phone number (or leave blank to finish): " ) ; 
		String number = stdin . readLine ( ) ; 
		if ( number . length ( ) == 0 ) { 
			break ; 
			} 
		Person . PhoneNumber . Builder phoneNumber = Person . PhoneNumber . newBuilder ( ) . setNumber ( number ) ; 
		stdout . print ( "Is this a mobile, home, or work phone? " ) ; 
		String type = stdin . readLine ( ) ; 
		if ( type . equals ( "mobile" ) ) { 
			phoneNumber . setType ( Per

In [44]:
tree_data[20572]

{'classBodyDeclaration': [{'modifier': [{'classOrInterfaceModifier': [{'type': 38,
       'text': 'static'}]}]},
  {'memberDeclaration': [{'methodDeclaration': [{'typeSpec': [{'classOrInterfaceType': [{'type': 100,
           'text': 'Person'}]}]},
      {'type': 100, 'text': 'PromptForAddress'},
      {'formalParameters': [{'type': 57, 'text': '('},
        {'formalParameterList': [{'formalParameter': [{'typeSpec': [{'classOrInterfaceType': [{'type': 100,
                 'text': 'BufferedReader'}]}]},
            {'variableDeclaratorId': [{'type': 100, 'text': 'stdin'}]}]},
          {'type': 64, 'text': ','},
          {'formalParameter': [{'typeSpec': [{'classOrInterfaceType': [{'type': 100,
                 'text': 'PrintStream'}]}]},
            {'variableDeclaratorId': [{'type': 100, 'text': 'stdout'}]}]}]},
        {'type': 58, 'text': ')'}]},
      {'type': 45, 'text': 'throws'},
      {'qualifiedNameList': [{'qualifiedName': [{'type': 100,
           'text': 'IOException'}]}]

In [ ]:
for key in abstract_code_keys:
    if "$id0 ( )" in key:
        print(key, abstract_code_dict[key]['count'])